<center>
        <div style="font-size: 40px; color: black"><b>Extraction of 3D information from binocular images</b></div>
</center>

EPPS Training School 12.09.2025

Samuel Thomas - R&D signal & image processing engineer - team Cigale, Arvalis (s.thomas@arvalis.fr)
Part of the LPA CAPTE

licensed under CC By NC 4.0

# Part 1 : Camera calibration

In this notebook, you will find the code for the 2 steps required before thinking retrieving the 3D information from a stereo camera pair :

* Calibration of a single camera

* Calibration of the stereo pair formed by two calibrated cameras



### 📌 Documentation can be found here :

From OpenCV :

[calibrateCamera](./documentation/OpenCV_calibrateCamera.pdf)

[stereoCalibrate](./documentation/OpenCV_stereoCalibrate.pdf)

https://docs.opencv.org/4.x/d6/d55/tutorial_table_of_content_calib3d.html

https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html

https://nikolasent.github.io/computervision/opencv/calibration/2024/12/20/Practical-OpenCV-Refinement-Techniques.html

In [ ]:
import glob as glob
import os

import numpy as np
import cv2 as cv

import calibrationlib

## Preparating data, paths, and variables

This calibration operation has theoritically to be done once, <u>if the cameras are securely fixed together</u> /!\

A minimal set of ~ 10 pairs of <u>perfectly synchronized</u> chessboard images is required officially /!\

In practice we take around 50 images, trying to cover the corners of the image, different orientations of the chessboard and a range of distances to the sensor, consistent with our applications.

The more images are well-exposed (taken outside for example), the less noise you will have and the better the chessboard detection will work.

Example of such images :

<img src="static/raw_chessboard.png" width="800"/>

In [ ]:
path_calibration = (r"./data/calibration/classical_system")

# Calibration paths for left ('Camera1') and right ('Camera2') chessboard images
path_images = [
    path_calibration + "/images/Left",
    path_calibration + "/images/Right",
]

path_matrices = path_calibration + "/example_matrices"
if os.path.isdir(path_matrices) is False:
    os.mkdir(path_matrices)

path_matrices_single_cam = [
    path_matrices + "/Left",
    path_matrices + "/Right",
]

if os.path.isdir(path_matrices_single_cam[0]) is False:
    os.mkdir(path_matrices_single_cam[0])

if os.path.isdir(path_matrices_single_cam[1]) is False:
    os.mkdir(path_matrices_single_cam[1])

image_type = "jpg"

# Chessboard pattern size (width, height)
chessboard_pattern = (13, 8)
# Chessboard pattern cell size (here in mm) : used for scaling calibration parameters
chessboard_cell_size = 40

calibration = calibrationlib.CameraCalibration(chessboard_pattern, chessboard_cell_size)

count = 0

object_points_cam1 = []
image_points_cam1 = []
object_points_cam2 = []
image_points_cam2 = []

success_images_cam1 = []
success_images_cam2 = []

name = path_calibration + "/errors.log"
log = open(name, "w")

## Step 1 : Calibration of a single camera

### The aim of this calibration is to get 2 essential parameters :

- <u>The intrinsic matrix M :</u>

<img src="static/intrinsic_matrix.png" width="150"/>

This matrix allows to project the 3D point coordinates (X,Y,Z) (in meters) into the 2D image plane (x,y) (in pixels)

<img src="static/_2d_projection.png" width="200"/>
    
- <u>The distorsion coefficients vector :</u>

<img src="static/dist_coefs.png" width="250"/>

These coefficients allows to correct the distorsions due to the lens and the manufacturing defaults

<img src="static/undistort_chessboard.png" width="700"/>

## 1. Calibration of the first (left) camera

In [ ]:
log_line = "Calibration - Left/first camera\r"
log.write(log_line)
print(log_line)

ret1 = False

(
    ret1,
    reprojection_error_cam1,
    success_images_cam1,
    roi_cam1,
    camera_matrix_cam1,
    dist_coefs_cam1,
    r_vecs_cam1,
    t_vecs_cam1,
    object_points_cam1,
    image_points_cam1,
    image_size_cam1,
) = calibration.single_calibration(
    path_images[0],
    image_type,
    log_file=log,
)

tmp = np.array(object_points_cam1)
name = path_matrices_single_cam[0] + "/object_points"
np.save(name, tmp)
tmp = np.array(image_points_cam1)
name = path_matrices_single_cam[0] + "/image_points"
np.save(name, tmp)
tmp = np.array(success_images_cam1)
name = path_matrices_single_cam[0] + "/success_images"
np.save(name, tmp)

tmp = np.array(image_size_cam1)
name = path_matrices_single_cam[0] + "/image_size"
np.save(name, tmp)

tmp = np.array(camera_matrix_cam1)
name = path_matrices_single_cam[0] + "/camera_matrix"
np.save(name, tmp)
tmp = np.array(dist_coefs_cam1)
name = path_matrices_single_cam[0] + "/dist_coefs"
np.save(name, tmp)
tmp = np.array(r_vecs_cam1)
name = path_matrices_single_cam[0] + "/r_vecs"
np.save(name, tmp)
tmp = np.array(t_vecs_cam1)
name = path_matrices_single_cam[0] + "/t_vecs"
np.save(name, tmp)
tmp = np.array(reprojection_error_cam1)
name = path_matrices_single_cam[0] + "/reprojection_error"
np.save(name, tmp)

log_line = f"End of left/first camera calibration successful : {ret1}\r"
log.write(log_line)
print(log_line)

### Outputs :

It's important to check the quality of the operation performed by the algorithm ; for this purpose, we can draw the corners that were detected on the original and quickly have a look at the results

<img src="static/corners_chessboard.png" width="800"/>

Then we can have a look to the intrinsic parameters and distorsion coefficients that were computed

Left camera matrix (3x3) :

In [ ]:
camera_matrix_cam1

Left distorsion coefficients vector (5x1) :

In [ ]:
dist_coefs_cam1

## Step 2 - Part 2 : Calibration of the second (right) camera

In [ ]:
# Right/second camera
log_line = "Calibration - Right/second camera\r"
log.write(log_line)
print(log_line)

ret2 = False

(
    ret2,
    reprojection_error_cam2,
    success_images_cam2,
    roi_cam2,
    camera_matrix_cam2,
    dist_coefs_cam2,
    r_vecs_cam2,
    t_vecs_cam2,
    object_points_cam2,
    image_points_cam2,
    img_size_cam2,
) = calibration.single_calibration(
    path_images[1],
    image_type,
    log_file=log,
)

tmp = np.array(object_points_cam2)
name = path_matrices_single_cam[1] + "/object_points"
np.save(name, tmp)
tmp = np.array(image_points_cam2)
name = path_matrices_single_cam[1] + "/image_points"
np.save(name, tmp)
tmp = np.array(success_images_cam2)
name = path_matrices_single_cam[1] + "/success_images"
np.save(name, tmp)

tmp = np.array(image_size_cam1)
name = path_matrices_single_cam[1] + "/image_size"
np.save(name, tmp)

tmp = np.array(camera_matrix_cam2)
name = path_matrices_single_cam[1] + "/camera_matrix"
np.save(name, tmp)
tmp = np.array(dist_coefs_cam2)
name = path_matrices_single_cam[1] + "/dist_coefs"
np.save(name, tmp)
tmp = np.array(r_vecs_cam2)
name = path_matrices_single_cam[1] + "/r_vecs"
np.save(name, tmp)
tmp = np.array(t_vecs_cam2)
name = path_matrices_single_cam[1] + "/t_vecs"
np.save(name, tmp)
tmp = np.array(reprojection_error_cam2)
name = path_matrices_single_cam[1] + "/reprojection_error"
np.save(name, tmp)

log_line = f"End of right/second camera calibration successfully achieved : {ret2}\r"
log.write(log_line)
print(log_line)

### Outputs :

Here we can do the same check as for the left camera

Right camera matrix (3x3) :

In [ ]:
camera_matrix_cam2

Right distorsion coefficients vector (5x1) :

In [ ]:
dist_coefs_cam2

## Step 2 : Calibration of the stereo pair

At this step, the two cameras of our stereo pair are calibrated so that each single camera geometry is well-known ; we now need to characterized the geometry of the stereo imaging system.
This consists in computing two main parameters : the rotation matrix and the translation that perform a change of basis from the first camera's coordinate system to the second camera's coordinate system.

These two parameters allow to correct the geometry of the system so that we are in the ideal case of two coplanar imager planes.

<img src="static/stereo_calibration.png" width="500"/>


In [ ]:
log_line = "Stereo calibration between left/first and right/second cameras\r"
log.write(log_line)
print(log_line)

if (ret1 is True) and (ret2 is True):

    # We keep only image pairs on which were successfully detected the chessboard corners
    # for both cameras, and store relative objects and image points for further stereo
    # calibration
    common_images, index_points_cam1, index_points_cam2 = np.intersect1d(success_images_cam1, success_images_cam2, return_indices=True)

    log_line = f"Common images : {(np.array(success_images_cam1))[index_points_cam1]}\r"
    log.write(log_line)
    log_line = f"Images indexes for camera 1 : {list(index_points_cam1)}\r"
    log.write(log_line)
    print(log_line)
    log_line = f"Images indexes for camera 2 : {list(index_points_cam2)}\r"
    log.write(log_line)
    print(log_line)


    # Resulting list of common points for the two cameras (i.e. from the image pairs on which
    # the chessboard has been successfully deteted in both cameras
    filtered_object_points_cam1 = np.array(object_points_cam1)
    filtered_object_points_cam1 = filtered_object_points_cam1[index_points_cam1]
    filtered_object_points_cam1 = list(filtered_object_points_cam1)
    
    filtered_image_points_cam1 = np.array(image_points_cam1)
    filtered_image_points_cam1 = filtered_image_points_cam1[index_points_cam1]
    filtered_image_points_cam1 = list(filtered_image_points_cam1)
    
    filtered_image_points_cam2 = np.array(image_points_cam2)
    filtered_image_points_cam2 = filtered_image_points_cam2[index_points_cam2]
    filtered_image_points_cam2 = list(filtered_image_points_cam2)

    # We compute now stereo calibration : camera_matrices and distorsion coefficients for each
    # camera may be slightly modified (depending on the flags parameters : in our case, the output 
    # values are strictly the same as the input ones), and we get the rotation matrix R and 
    # translation matrix T that bring points in the left camera coordinate system to points in the 
    # right camera coordinate system (you may read the doc : documentation/OpenCV_stereoCalibrate) 
    (
        ret,
        camera_matrix_cam1_stereo,
        dist_coefs_cam1_stereo,
        camera_matrix_cam2_stereo,
        dist_coefs_cam2_stereo,
        R,
        T,
    ) = calibration.stereo_calibration(
        filtered_object_points_cam1,
        filtered_image_points_cam1,
        filtered_image_points_cam2,
        camera_matrix_cam1,
        dist_coefs_cam1,
        camera_matrix_cam2,
        dist_coefs_cam2,
        image_size_cam1,
        log_file=log,
    )

    # We save the useful parameters, from which further rectification of images will 
    # be possible for matching. The rectification maps will be loaded once at the 
    # beginning of the matching code, and applied to each images pair.
    tmp = np.array(camera_matrix_cam1_stereo)
    name = path_matrices_single_cam[0] + "/camera_matrix_from_stereo"
    np.save(name, tmp)
    tmp = np.array(dist_coefs_cam1_stereo)
    name = path_matrices_single_cam[0] + "/dist_coefs_from_stereo"
    np.save(name, tmp)
    tmp = np.array(camera_matrix_cam2_stereo)
    name = path_matrices_single_cam[1] + "/camera_matrix_from_stereo"
    np.save(name, tmp)
    tmp = np.array(dist_coefs_cam2_stereo)
    name = path_matrices_single_cam[1] + "/dist_coefs_from_stereo"
    np.save(name, tmp)
    tmp = np.array(R)
    name = path_calibration + "/R"
    np.save(name, tmp)
    tmp = np.array(T)
    name = path_calibration + "/T"
    np.save(name, tmp)
    tmp = np.array(ret)
    name = path_calibration + "/reprojection_error"
    np.save(name, tmp)

    log_line = "End of stereo calibration successfully achieved\r"
    log.write(log_line)
    print(log_line)
    
else:
    log_line = "Problem with the calibration of one of the two cameras - no way to perform stereo calibration"
    log.write(log_line)
    print(log_line)

log_line = "End of calibration process"
log.write(log_line)
print(log_line)

### Outputs :

Then we can have a look to the rotation matrix R and the translation vector that were computed

R (3x3) matrix

In [ ]:
R

T (1x3) vector

In [ ]:
T

Question : what basic but essential information involved in calculating depth do we now have here ?
How to access it as a single value ?